# Data Retrieval for Machine Learning

In this notebook, we retrieve MLB data from 1996-2017 that we will use to fit a machine learning model to. We decided that we wanted to see if we could find a model that would predict team wins even better than the Pythagorean Expectation does. After researching and examining baseball statistics, we decided to not only focus on team statistics as a whole, but pitcher statistics as well. The performance of the pitcher has a huge impact on the game results. 

In [1]:
import numpy as np
import pandas as pd
import requests
import plotly.offline as py
import matplotlib.pyplot as plt
from plotly.graph_objs import *
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import cross_val_score
from sklearn.cluster import KMeans
from bs4 import BeautifulSoup
%matplotlib inline
pd.set_option("max_r", 15)
py.init_notebook_mode(connected=True)

For unknown reasons, the seventh and eighth tables that we need from the above URLs do not appear in the html returned from BeautifulSoup (it skips over them). Thus, instead of scraping from them like we did for the other datasets we need in the previous notebook, we are taking CSV's. It is also worth noting that this is the only website that has the advanced metrics we need for our calculations, which makes this situation even more of a shame.

In [2]:
# read in all the csvs
dfs = []
for i in range(1996, 2017):
    fname = "mlb%d.csv" % i
    df = pd.read_csv(fname)
    df["year"] = i
    dfs.append(df)

In [3]:
# concat all the team data into one
mlb = pd.concat(dfs).dropna()
mlb = mlb.sort_values(by=["Tm","year"])
mlb.head()

,Rk,Tm,Lg,G,W,L,W-L%,R,RA,Rdiff,...,Inter,Home,Road,ExInn,1Run,vRHP,vLHP,≥.500,<.500,year
10,11.0,ANA,AL,162.0,84,78,0.518,5.1,4.9,0.2,...,4-12,46-36,38-42,5-8,27-25,66-55,18-23,23-34,61-44,1997
11,12.0,ANA,AL,162.0,85,77,0.525,4.9,4.8,0.0,...,10-6,42-39,43-38,6-6,24-19,62-59,23-18,30-36,55-41,1998
22,23.0,ANA,AL,162.0,70,92,0.432,4.4,5.1,-0.7,...,6-12,37-44,33-48,4-8,23-25,51-73,19-19,27-45,43-47,1999
14,15.0,ANA,AL,162.0,82,80,0.506,5.3,5.4,0.0,...,12-6,46-35,36-45,9-7,32-23,61-55,21-25,42-49,40-31,2000
19,20.0,ANA,AL,162.0,75,87,0.463,4.3,4.5,-0.2,...,10-8,39-42,36-45,7-8,24-24,53-60,22-27,37-55,38-32,2001


In [4]:
# read in pitching data
dfs = []
for i in range(1996, 2017):
    fname = "pitch%d.csv" % i
    df = pd.read_csv(fname)
    df["year"] = i
    dfs.append(df)

In [5]:
# combine pitching data 
pitch = pd.concat(dfs).dropna()
pitch = pitch.sort_values(by=["Tm","year"])
pitch.head()

,Tm,#P,PAge,RA/G,W,L,W-L%,ERA,G,GS,...,ERA+,FIP,WHIP,H9,HR9,BB9,SO9,SO/W,LOB,year
0,ANA,21,28.3,4.90,84,78,0.519,4.52,162,162,...,104,4.83,1.451,9.3,1.2,3.7,6.5,1.74,1207,1997
0,ANA,19,29.8,4.83,85,77,0.525,4.49,162,162,...,106,4.51,1.462,9.2,1.0,3.9,6.8,1.73,1211,1998
0,ANA,20,31.5,5.10,70,92,0.432,4.79,162,162,...,101,4.94,1.464,9.3,1.1,3.9,5.5,1.41,1138,1999
0,ANA,25,28.9,5.36,82,80,0.506,5.00,162,162,...,101,5.46,1.517,9.5,1.4,4.1,5.3,1.28,1188,2000
0,ANA,18,28.9,4.51,75,87,0.463,4.20,162,162,...,107,4.48,1.375,9.1,1.1,3.3,5.9,1.80,1152,2001


In [6]:
mlb = mlb.reset_index().drop("index",axis=1)
pitch = pitch.reset_index().drop("index",axis=1)

In [7]:
# Ensure all overlapping columns have the same values.
overlapping_cols = ['Tm', 'W', 'L', 'G', 'year']
df = (mlb[overlapping_cols] == pitch[overlapping_cols])
df.all()

Tm      True
W       True
L       True
G       True
year    True
dtype: bool

In [8]:
# Convert runs and runs allowed to yearly numbers from averages per game.
mlb['R'] = mlb['R'] * 162
mlb['RA'] = mlb['RA'] * 162

In [9]:
# Add all of the columns we need into one dataframe.
mlb.head()

,Rk,Tm,Lg,G,W,L,W-L%,R,RA,Rdiff,...,Inter,Home,Road,ExInn,1Run,vRHP,vLHP,≥.500,<.500,year
0,11.0,ANA,AL,162.0,84,78,0.518,826.2,793.8,0.2,...,4-12,46-36,38-42,5-8,27-25,66-55,18-23,23-34,61-44,1997
1,12.0,ANA,AL,162.0,85,77,0.525,793.8,777.6,0.0,...,10-6,42-39,43-38,6-6,24-19,62-59,23-18,30-36,55-41,1998
2,23.0,ANA,AL,162.0,70,92,0.432,712.8,826.2,-0.7,...,6-12,37-44,33-48,4-8,23-25,51-73,19-19,27-45,43-47,1999
3,15.0,ANA,AL,162.0,82,80,0.506,858.6,874.8,0.0,...,12-6,46-35,36-45,9-7,32-23,61-55,21-25,42-49,40-31,2000
4,20.0,ANA,AL,162.0,75,87,0.463,696.6,729.0,-0.2,...,10-8,39-42,36-45,7-8,24-24,53-60,22-27,37-55,38-32,2001


The variables that we felt would be most usuful for predicting wins with a machine learning model were:
+ Runs: the total number of runs the team scored during the season
+ Runs Allowed: the total number of runs that were scored against the team during the season
+ Strength of Schedule: the difficulty or ease of a team's opponent as compared to other teams.
+ ERA: the mean of the earned runs given up by a pitcher per nine innings pitched
+ WHIP: walks hits per inning pitched
+ FIP: field independent pitching
+ SO: strike outs

In [10]:
# Create a dataframe for our training data.
machine_data = pd.DataFrame(mlb[['R',"RA","SOS",'W',"year"]])
machine_data['ERA'] = pitch['ERA']
machine_data['WHIP'] = pitch['WHIP']
machine_data['FIP'] = pitch['FIP']
machine_data['SO'] = pitch['SO']

In [11]:
machine_data.dtypes

R       float64
RA      float64
SOS     float64
W         int64
year      int64
ERA     float64
WHIP    float64
FIP     float64
SO        int64
dtype: object

In [12]:
machine_data.to_csv("training.csv", index=False)